In [1]:
import numpy as np
import pandas as pd
from numpy import sqrt,arctan,arccos,arcsin,sin,cos,tan,mod,pi,mean

In [2]:
def deg2rad(x):
    return x*pi/180
def rad2deg(x):
    return x*180/pi

# Wrap angle between (-180,+180)
def wrap_angle(theta):
    angle = deg2rad(theta)
    angle = mod(angle,2*pi)
    if angle > pi:
        angle = angle - 2*pi
    return rad2deg(angle)

# Unwrap angle to (0,360)
def unwrap_angle(theta):
    angle = deg2rad(theta)
    if angle < 0.:
        angle = pi - angle
    return rad2deg(angle)

In [3]:
def get_cart(lon,lat,r=1):
#    if lat<-90 or lat>+90:
#        raise Exception("-90 <= latitude <= +90")
    phi = pi/2.-deg2rad(float(lat))
    theta = deg2rad(wrap_angle(float(lon)))
    x = r*sin(phi)*cos(theta)
    y = r*sin(phi)*sin(theta)
    z = r*cos(phi)
    return (x,y,z)

def get_sph(x,y,z):
    r = sqrt(x*x+y*y+z*z)
    phi = arccos(z/r)
    theta = arcsin(y/r/sin(phi))
    lon = unwrap_angle(rad2deg(theta))
    lat = rad2deg(pi/2.-phi)
    if lat > 90.:
        lat = lat-180.
    return (r,lon,lat)
    

In [4]:
def get_centeroid(arr):
    npts = 0
    xc,yc,zc = 0.,0.,0.
    for i in range(0,len(arr)):
        cart = get_cart(arr["GL"][i],arr['GB'][i],r=arr['r'][i])
        xc += cart[0]
        yc += cart[1]
        zc += cart[2]
        npts+=1
    out = get_sph(xc/npts,yc/npts,zc/npts) 
    return out

In [5]:
def get_dist(P1,P2):
    (x1,y1,z1) = get_cart(P1["GL"],P1["GB"],P1["r"])
    (x2,y2,z2) = get_cart(P2["GL"],P2["GB"],P2["r"])
    dist = sqrt((x2-x1)**2+(y2-y1)**2+(z2-z1)**2)
    return rad2deg(dist)

In [6]:
gal = {"GL":[248,235.2,240,257.1,237.4,266.2,261.3],
       "GB":[-49.2,-49.1,-48.6,-48.4,-48.2,-44.4,-40.2],
       "r":[1,1,1,1,1,1,1]}
arr = pd.DataFrame.from_dict(gal)
df = arr.copy().reset_index()
print(df)
cent = get_centeroid(df)
print(f"centeroid {cent}")

   index     GL    GB  r
0      0  248.0 -49.2  1
1      1  235.2 -49.1  1
2      2  240.0 -48.6  1
3      3  257.1 -48.4  1
4      4  237.4 -48.2  1
5      5  266.2 -44.4  1
6      6  261.3 -40.2  1
centeroid (0.9891518159813015, 249.7646203207101, -47.45592677277548)


In [10]:
C = pd.DataFrame.from_dict([{"GL":cent[1],"GB":cent[2],"r":cent[0]}])
df1 = arr.copy().reset_index()
df1["dist"]=[0,0,0,0,0,0,0]

In [28]:
for i in range(len(df1)):
    P = df1.iloc[[i]]
    P = P.copy().reset_index()
    dist = get_dist(P,C)
    df1.loc[[i],"dist"] = [dist]

In [29]:
df1

,index,GL,GB,r,dist
0,0,248.0,-49.2,1,2.180739
1,1,235.2,-49.1,1,9.769989
2,2,240.0,-48.6,1,6.614480
3,3,257.1,-48.4,1,5.012840
4,4,237.4,-48.2,1,8.296035
5,5,266.2,-44.4,1,11.739586
6,6,261.3,-40.2,1,10.960849


In [36]:
max_dist = max(df1.dist)
base = round(7.)
dist_round = base*round(max_dist/base)
if dist_round < max_dist:
    dist_round+=base
print(dist_round)

14
